<a href="https://colab.research.google.com/github/RubeRad/StyleGAN2-TensorFlow-2.x/blob/master/StyleGAN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import the normal stuff
import tensorflow        as tf
import numpy             as np
import matplotlib.pyplot as plt
import os

In [ ]:
# DO THIS IF YOU ARE IN COLAB
# Clone the repository with the TensorFlow2 update to StyleGAN2
# Thanks Alberto Rosas!!
%cd /content
!git clone https://github.com/ruberad/StyleGAN2-TensorFlow-2.x.git stylegan2
%cd /content/stylegan2

In [ ]:
# import StyleGAN2-specific stuff
from utils.utils_stylegan2 import convert_images_to_uint8
from stylegan2_generator   import StyleGan2Generator

In [ ]:
# Check if you have access to a GPU in this (virtual) machine
!nvidia-smi -L
gpuname = tf.test.gpu_device_name()
print('GPU Identified at: "{}"'.format(gpuname))

if gpuname:
  impl = 'cuda'
  gpu = True
  print('yay fast!')
else:
  impl = 'ref'
  gpu = False
  print('aww, slow.')

In [ ]:
import gdown
# This is the public URL of the online file with weights that StyleGan2 needs for generating faces
url = 'https://drive.google.com/uc?export=download&confirm=pbef&id=1afMN3e_6UuTTPDL63WHaA0Fb9EQrZceE'
# This is where we want the file to go (path on the virtual machine)
out = 'weights/ffhq.npy'

if os.path.exists(out):
  print('ffhq.npy weights file is present')
else:
  gdown.download(url, out, quiet=False)

In [ ]:
weights_name='ffhq' # face generation, high quality
sg2 = StyleGan2Generator(weights=weights_name, impl=impl, gpu=gpu)

In [ ]:
seed = 1
rng = np.random.RandomState(seed)

In [ ]:
z = rng.randn(1, 512).astype('float32')
dlatents = sg2.mapping_network(z)

trunc = 0.7
w_avg = sg2.dlatent_avg
dlatents = w_avg + (dlatents - w_avg) * trunc

out = sg2.synthesis_network(dlatents)

img = convert_images_to_uint8(out, nchw_to_nhwc=True, uint8_cast=True)


In [ ]:
#plotting images
fig = plt.figure()
ax = plt.gca()
ax.axis('off')
img_plot = ax.imshow(img.numpy()[0])